In [25]:
!pip install -q -U google-generativeai

In [26]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

In [38]:
# Import necessary libraries
import pandas as pd
from google.colab import files

# Step 1: Upload the Excel files
uploaded = files.upload()

In [39]:
# Load the uploaded Excel files
cement_data = pd.read_excel('BEST_cement.xlsx')
news_data = pd.read_excel('financial_news.xlsx')

# Display the first few rows of each dataset
print("Cement Data:")
print(cement_data)

print("\nFinancial News Data:")
print(news_data.head())

Cement Data:
          Date   Price    Open    High     Low    Vol.  Change %
0   2020-11-02  160.50  160.25  163.00  159.00  35.40K   -0.0273
1   2020-11-03  163.15  161.00  164.99  160.00  27.30K    0.0165
2   2020-11-04  162.50  165.00  167.00  162.05  24.40K   -0.0040
3   2020-11-05  167.00  164.99  170.00  162.00  53.10K    0.0277
4   2020-11-06  165.72  165.51  166.50  165.00  17.30K   -0.0077
..         ...     ...     ...     ...     ...     ...       ...
198 2021-08-25  154.50  155.00  156.21  154.00   6.30K   -0.0032
199 2021-08-26  154.00  155.50  155.50  153.22   6.40K   -0.0032
200 2021-08-27  152.55  153.03  153.03  152.55   1.60K   -0.0094
201 2021-08-30  156.00  154.00  157.98  154.00  18.80K    0.0226
202 2021-08-31  156.20  156.00  156.60  156.00   9.70K    0.0013

[203 rows x 7 columns]

Financial News Data:
                                            Headline   DateTime  \
0                  Pharma industry's forex potential 2020-11-01   
1  Centre to grant status o

In [40]:
# Define the new date range
start_date = pd.Timestamp('2021-02-01')
end_date = pd.Timestamp('2021-03-31')

# Step 1: Filter cement_data to include only dates within the new range
filtered_cement_data = cement_data[(cement_data['Date'] >= start_date) & (cement_data['Date'] <= end_date)]

# Step 2: Find significant dates where the change is > 3 or < -3 within the filtered range
significant_dates = filtered_cement_data[(filtered_cement_data['Change %'] > 0.03) | (filtered_cement_data['Change %'] < -0.03)]['Date'].unique()

# Convert significant dates to x/y/z format
significant_dates_formatted = [date.strftime('%m/%d/%Y') for date in significant_dates]

# Find row numbers for significant dates
row_numbers = []
for date in significant_dates:
    rows = filtered_cement_data[filtered_cement_data['Date'] == date].index.tolist()
    row_numbers.extend(rows)

print("Deliverable 1: List of Key Dates")
print("Identified dates with significant market variations.")

# Display the formatted significant dates and their row numbers
print("\nSignificant Dates (x/y/z format) and Row Numbers:")
for date in significant_dates_formatted:
    print(f"Date: {date}, Row Numbers: {filtered_cement_data[filtered_cement_data['Date'].dt.strftime('%m/%d/%Y') == date].index.tolist()}")


Deliverable 1: List of Key Dates
Identified dates with significant market variations.

Significant Dates (x/y/z format) and Row Numbers:
Date: 02/10/2021, Row Numbers: [70]
Date: 03/04/2021, Row Numbers: [86]
Date: 03/05/2021, Row Numbers: [87]
Date: 03/08/2021, Row Numbers: [88]
Date: 03/29/2021, Row Numbers: [102]


In [56]:
# Create a DataFrame from the significant dates
df_significant_dates = pd.DataFrame(significant_dates_formatted, columns=['Significant Dates'])

# Save the DataFrame to a CSV file
df_significant_dates.to_csv('final_dates.csv', index=False)

# Download the CSV file
files.download('final_dates.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
# Convert 'DateTime' column to datetime format if not already done
news_data['DateTime'] = pd.to_datetime(news_data['DateTime'])

# Initialize an empty DataFrame to collect the filtered news articles
filtered_news = pd.DataFrame()

# For each significant date, include that date and the two days before
for date in significant_dates:
    # Generate the date range including the date and 2 days before
    date_range = pd.date_range(start=date - pd.Timedelta(days=2), end=date)

    # Filter news_data for these dates
    news_subset = news_data[news_data['DateTime'].isin(date_range)]

    # Append to the filtered news DataFrame
    filtered_news = pd.concat([filtered_news, news_subset])

# Remove duplicates if any
filtered_news = filtered_news.drop_duplicates()

print("Deliverable 2: Collected News Articles")
print("A set of news articles from the selected dates and two days prior.")

# Display the filtered news DataFrame
print("\nFiltered News Data:")
print(filtered_news)


Deliverable 2: Collected News Articles
A set of news articles from the selected dates and two days prior.

Filtered News Data:
                                               Headline   DateTime  \
3158                   Are GDP growth prospects bright? 2021-02-08   
3159                     Making sense of inflation data 2021-02-08   
3160  NFC Ã¯Â¿Â½Ã¯Â¿Â½Ã¯Â¿Â½Ã¯Â¿Â½Ã¯Â¿Â½Ã¯Â¿Â½Ã¯Â¿Â½... 2021-02-08   
3161  Alyani accuses previous govts of misusing upli... 2021-02-08   
3162  Sardar Masood Khan invites investors to tap in... 2021-02-08   
...                                                 ...        ...   
4847  Rupee gains another 42 paisa as dollar falls t... 2021-03-29   
4848               Rupee gains 55 paisas against dollar 2021-03-29   
4849  Sri Lankan shares end higher on industrial, fi... 2021-03-29   
4850               Digital banks: still a distant dream 2021-03-29   
4851  Saudi govt to emulate PTIÃ¯Â¿Â½Ã¯Â¿Â½Ã¯Â¿Â½Ã¯Â... 2021-03-29   

                 Source         

In [54]:
from google.colab import files

# Save the filtered news DataFrame to a CSV file
filtered_news.to_csv('filtered_news.csv', index=False)

# Download the CSV file
files.download('filtered_news.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
# Extract only the "News" column from the filtered_news DataFrame
news_only = filtered_news['News'].reset_index(drop=True)

# Display the extracted news
print(news_only)



0      Despite many positives, LNG crisis and fiscal ...
1      Three economic things can add to or subtract f...
2      The federal government has transferred Rs1.28 ...
3      QUETTA: Balochistan Chief Minister Jam Kamal K...
4      ISLAMABAD: Azad Jammu and Kashmir (AJK) Presid...
                             ...                        
387    The rupee picked up where it left off on Frida...
388    ISLAMABAD: The exchange rate of Pakistan rupee...
389    Sri Lankan shares ended higher on Monday, help...
390    While much of the country, particularly prime-...
391    ISLAMABAD: The Saudi government has announced ...
Name: News, Length: 392, dtype: object


In [44]:
# Now you can safely access the first element
print(news_only[0])

Despite many positives, LNG crisis and fiscal deficit can decelerate economic progress
KARACHI:
 
 The  Asian Development Bank  (ADB) says PakistanÃƒÂ¢Ã‚Â€Ã‚Â™s economy can grow 2% in the current fiscal year ending June 2021 after contracting 0.4% last year. 
 The government has also set almost a similar growth target at 2.1%. But we can safely expect a higher growth rate. 
 The most important driver for a higher-than-expected growth is the pace at which global economy is going to recover in 2021. The International Monetary Fund (IMF) has forecast 5.2% and Morgan Stanley 6.4% global growth. The IMF makes such projections very cautiously and takes into account the behaviour of all variables. 
 However, the IMF has clarified that its forecast rests not only on economic factors but also on public health ÃƒÂ¢Ã‚Â€Ã‚Âœwhich is inherently difficult to predictÃƒÂ¢Ã‚Â€Ã‚Â. So, we can expect that global  economic growth  this year could vary vastly from the numbers being projected right now. 
 

https://chatgpt.com/share/f888050e-be32-4a55-a895-a0ffdd5bda19

https://chatgpt.com/share/5027609f-5cef-4d1b-9177-e8097564d767


In [53]:
import pandas as pd
from google.colab import files

# Define the list of news articles
data = [
    ["Pakistan has offered Saudi government technical support for its implementation of the programme", "Positive", "Technical support for plantation projects might create opportunities for cement companies involved in providing infrastructure or materials", 0.2],
    ["Rupee strengthened by 55 paisas against the US dollar", "Positive", "A stronger rupee can reduce the cost of imported raw materials and equipment for cement manufacturers, potentially benefiting the sector", 0.3],
    ["Dollar closed at Rs154.03 compared to Rs154.58 previously", "Positive", "A lower dollar value can decrease the cost of imports for the cement industry, positively impacting their profitability", 0.3],
    ["Euro depreciated by 84 paisas to Rs181.24", "Positive", "A weaker euro can lower costs for cement companies importing from Eurozone countries, potentially benefiting the sector", 0.3],
    ["Japanese Yen lost one paisa to close at Rs1.40", "Positive", "A weaker yen can reduce costs for cement companies importing from Japan, benefiting the sector", 0.2],
    ["British Pound decreased by 17 paisas to Rs212.78", "Positive", "A weaker pound can lower costs for imports from the UK, potentially benefiting the cement industry", 0.2], ["Rupee gained an additional 42 paisa against the US dollar", "Positive", "A stronger rupee can reduce the cost of imported raw materials and equipment for cement manufacturers, potentially benefiting the sector", 0.3], ["Dollar quoted at below the pre-pandemic level of 154.24", "Positive", "A lower dollar value can lead to reduced costs for cement companies importing materials, positively impacting their profitability", 0.3], ["Dollar has depreciated 9.1 percent against the rupee since July 1, 2020", "Positive", "Significant depreciation of the dollar can lower the cost of imports for the cement industry, potentially benefiting cement manufacturers", 0.3], ["NSE Nifty 50 index ended 1.27% higher", "Positive", "An increase in the NSE Nifty 50 index indicates a general improvement in market conditions, which could positively influence investor sentiment towards sectors including cement", 0.2], ["S&P BSE Sensex rose 1.17%", "Positive", "A rise in the S&P BSE Sensex suggests broader market gains, potentially benefiting sectors such as cement through improved market confidence", 0.2], ["10-year benchmark bond yield ended at 6.12%", "Negative", "Higher bond yields can lead to increased borrowing costs for cement companies, potentially impacting their profitability and investment plans", 0.2], ["KSE-100 index lost 1089.83 points, closing at 44,431.80 points", "Negative", "A significant decline in the KSE-100 index reflects overall market weakness, which can negatively impact investor sentiment towards the cement sector", 0.2], ["Price of shares stood at Rs23.514 billion against Rs25.185 billion previous day", "Negative", "A drop in the total value of shares traded indicates reduced market activity and investor confidence, which could affect the cement sector's stock performance", 0.2], ["LHC division bench seeks para-wise comments by April 14 from NAB on Khawaja Asif's post-arrest bail", "Negative", "Legal uncertainties and high-profile cases can impact market sentiment, including the cement sector, as they may affect investor confidence and regulatory stability", 0.1], ["Controversial Ravi Riverfront Urban Project", "Negative", "The acquisition of land for this project may indicate increased construction activity, potentially leading to higher demand for cement, though the controversy and legal issues could create uncertainty in the sector", 0.4], ["Government has acquired land in Bokanwala, Kherdinpur, Chohan, Kachi Kotli, Qadian, Rasoolpur, Dhanoya, Shediwal, Ratniwal, Babakwal, Nehraypur, Jada and Dandian villages", "Positive", "Increased land acquisition for urban development projects can lead to higher demand for construction materials like cement", 0.5], ["About 15 families live in Nehraypur, a small village whose all 63 acres have been acquired", "Positive", "The acquisition of land for large projects can lead to increased cement demand for new construction", 0.4], ["The total length of the project is 46km, all along the Ravi", "Positive", "A large-scale project like this typically requires substantial amounts of cement, potentially boosting the sector", 0.5], ["Not a single farmer is ready to give his land for the riverfront project", "Negative", "Opposition and legal challenges to the project could delay construction and reduce immediate demand for cement", 0.3], ["The government has cleverly taken greater part of land from Sheikhupura district", "Negative", "Displacement and legal disputes could cause uncertainty in the construction sector, potentially impacting cement demand and stock index", 0.4], ["Frequent glacial lake outbursts, floods, and other climate-related disasters", "Negative", "Increased frequency of climate-related disasters can disrupt supply chains and increase operational costs for cement manufacturers", 0.4], ["Locust storms destroying croplands in Sindh and Punjab", "Negative", "Destruction of croplands could impact the agricultural sector and indirectly affect cement demand by influencing infrastructure and construction projects", 0.3], ["Melting glaciers and rising sea levels impacting the Indus Delta", "Negative", "Environmental degradation and rising sea levels could disrupt construction projects and increase costs for cement manufacturers in affected regions", 0.4], ["The 10 billion tree project and the premier's announcement to generate 60% of energy from renewable resources by 2030", "Positive", "Initiatives to improve environmental conditions and renewable energy could lead to more sustainable practices in the cement industry, though the direct impact might be more gradual", 0.3], ["The state of investigation and prosecution in Pakistan remains in a shambles", "Negative", "Persistent issues in the criminal justice system could contribute to political and economic instability, potentially impacting business confidence and operations in sectors including cement", 0.3], ["Carjackings in Washington DC increased by 143% from the prior year", "Negative", "A significant rise in crime rates, including carjackings, could impact business operations and logistics, potentially disrupting the supply chain and affecting the cement sector's efficiency", 0.2], ["The blue-chip FTSE 100 index was down 0.4%, with bank stocks including HSBC Holdings, Barclays, and Prudential Financial Inc falling between 0.9% and 1.7%", "Negative", "The decline in the FTSE 100, which includes significant market players, could reflect broader market uncertainty that may indirectly affect investor sentiment in the cement sector", 0.1], ["The deal was received well by Wall Street, with BowX shares ending trade on Friday up 20% following the merger's announcement", "Positive", "Increased interest in BowX and positive market reception could indirectly impact investor sentiment in related sectors, including cement", 0.1], ["Wanda Light Asset Commercial Management Co, which is tasked with managing 368 existing Wanda Plazas and a further 155 under construction", "Positive", "The construction of 155 new Wanda Plazas could increase demand for cement, positively impacting the sector", 0.5], ["A multipurpose sports park would be built around the Quaid's portrait, which was re-erected on a hillock along the Express Highway near Koral Interchange, after its fencing", "Positive", "Development projects like parks can boost local construction activity, potentially benefiting cement demand", 0.4], ["State Bank of Pakistan (SBP) Deputy Governor Murtaza Syed said the immediate measures undertaken by the SBP, such as slashing the interest rate to 7% and financing wages and salaries of workers, helped the economy recover better than its global peers", "Positive", "Lower interest rates and financial support can reduce borrowing costs and enhance liquidity for the cement sector, potentially benefiting the industry", 0.6], ["The cement sector followed the same trend where Pioneer Cement (-5.8%), Cherat Cement (-5.3%), Maple Leaf Cement Factory (-3.7%), DG Khan Cement (-3.8%) and Lucky Cement (-2.3%) shed value", "Negative", "The decline in major cement stocks indicates a negative impact on the sector, likely driven by broader market trends and investor sentiment", 0.9], ["PM Imran Khan was determined to take on the mafias exploiting the masses", "Negative", "A crackdown on economic mafias may cause short-term instability in the market, which could indirectly affect the cement sector", 0.3], ["The construction industry would ensure improvement in the economy", "Positive", "Increased activity in the construction industry could drive higher demand for cement, benefiting the sector", 0.7], ["Government was taking concrete measures to provide a conducive atmosphere for international and national investors", "Positive", "A conducive investment environment could stimulate growth in construction projects and demand for cement", 0.5], ["CDNS has reached a fresh deposit of Rs 540 billion during the last seven months", "Positive", "Increased deposits could indicate a stable financial environment, potentially supporting economic growth and infrastructure projects that may benefit the cement industry", 0.3], ["CDNS has set Rs 249 billion annual collection target collection from July 1 to June 30 for the year 2020-21", "Positive", "A stable or increased collection target could reflect a robust financial system, potentially leading to greater infrastructure investment and higher demand for cement", 0.3], ["CDNS has opened new avenues for public and private investment to document the country’s economy and ensure transparency", "Positive", "Enhanced transparency and new investment avenues could stimulate economic activities and infrastructure projects, positively impacting cement demand", 0.3], ["Pakistan's economy has made significant gains towards stabilisation since 2018", "Positive", "Economic stabilisation can lead to increased infrastructure projects, boosting cement demand", 0.3], ["We shall continue to consolidate these gains and strengthen the growth momentum", "Positive", "Strengthening economic growth momentum can lead to higher construction activity, benefiting the cement industry", 0.3], ["We navigated economic fallout of covid-19 fairly well", "Positive", "Effective handling of economic fallout from COVID-19 may lead to economic recovery and increased construction activity, which could benefit the cement sector", 0.3], ["The world is witnessing supply chain disruptions & rise in food prices", "Negative", "Supply chain disruptions might affect cement production and distribution, potentially impacting the sector negatively", 0.3], ["We shall endeavour to protect our people from these shocks", "Positive", "Efforts to mitigate economic shocks could help stabilize the construction sector and demand for cement", 0.2], ["A nation like Pakistan could not depend on import and trading and the country needed to focus on manufacturing based on technology", "Positive", "Emphasis on manufacturing could increase demand for cement in construction and industrial activities", 0.3], ["A strong industrial base has to be anchored to agriculture, energy, food and financial security", "Positive", "Strengthening the industrial base could lead to increased infrastructure projects requiring cement", 0.4], ["Development experts recognise that the uneven growth of various sectors of the economy is a major cause of surging inequality in regional development and household incomes", "Negative", "Economic inequalities could affect regional development, potentially leading to uneven demand for cement", 0.3], ["There is strong opposition within the government as well as from common citizens against IMF policies", "Negative", "Potential for economic instability due to opposition to IMF policies could affect cement sector investments", 0.3], ["The corporate relief package has revived the formal economy", "Positive", "The revival of the formal economy could lead to increased construction activity, potentially boosting demand for cement", 0.3], ["Job creation in an underperforming economy is not just a social issue", "Negative", "An underperforming economy might lead to reduced construction projects and lower demand for cement", 0.2], ["The government needs to extend support to establishments operating under the radar", "Negative", "Limited support for informal sectors could slow down economic recovery, indirectly affecting cement demand", 0.2], ["For job creation, the government will need to extend support to small businesses", "Negative", "Limited support for small businesses might impact construction and infrastructure projects, potentially affecting cement demand", 0.2], ["The collapse of the parallel economy in the wake of the pandemic", "Negative", "Economic instability and collapse of businesses could lead to decreased construction activity and reduced cement consumption", 0.3], ["The poorest third of the population lost significant income due to reliance on the informal economy", "Negative", "Increased economic hardship could result in reduced construction activity, impacting cement demand", 0.2], ["Farmers cannot afford to switch to new technologies as they require heavy investment", "Negative", "Heavy investments required for new technologies can impact related industries, including cement, by reducing agricultural productivity and indirectly affecting the demand for cement in construction and infrastructure", 0.2], ["Government is working to facilitate the adoption of modern irrigation systems", "Positive", "Adoption of modern irrigation systems could lead to increased agricultural output, potentially driving higher demand for cement in related infrastructure projects", 0.3], ["Pedavar's solution of cultivating crops on raised beds and reducing water consumption", "Positive", "Effective agricultural techniques that enhance productivity may indirectly increase construction and infrastructure activities, potentially boosting demand for cement", 0.2], ["Some brands are keeping prices affordable while maintaining quality", "Positive", "Brands offering value for money may attract more price-sensitive customers", 0.3], ["Price hikes attributed to increased costs of fabric, yarn, dyes, and electricity", "Negative", "Rising input costs could further strain budgets and influence pricing strategies", 0.2], ["Luxury lawn brands are still selling well despite higher prices", "Positive", "Continued sales of high-priced items indicate strong brand loyalty and market demand", 0.3], ["Brands are using extensive advertising and social media campaigns", "Positive", "Increased marketing efforts suggest ongoing consumer interest and potential for sustained sales", 0.2], ["Luxury lawn prices have increased by 40% due to rising production costs", "Negative", "Higher production costs leading to increased prices might reduce affordability and impact sales volume", 0.2], ["Designer lawn market continues despite economic downturn and pandemic", "Positive", "Strong demand for luxury lawn despite economic challenges indicates resilience and potential growth in the textile sector", 0.3], ["Optimising public procurement could save at least $18 billion annually", "Positive", "Savings in procurement can potentially reduce costs for public sector projects including construction, indirectly benefiting cement sector demand", 0.3], ["Large and efficient public sectors in countries like Norway, Singapore, China, GCC member states", "Positive", "Examples of efficient public sectors highlight potential benefits of better procurement practices, which could improve infrastructure projects and cement demand", 0.3], ["Governance structure improvements in procurement could lead to significant savings", "Positive", "Improved governance and procurement processes can lead to cost reductions in public projects, potentially increasing demand for cement", 0.3], ["Increase in power tariff by Rs5.65 per unit", "Negative", "Higher power costs can increase operational expenses for cement manufacturers, potentially reducing profitability", 0.6], ["Gas utilities have asked for similar or even higher increase in their tariffs", "Negative", "Increased gas tariffs could raise production costs for cement manufacturers, leading to higher prices or reduced margins", 0.5], ["Reduce power tariff, electricity demand will increase", "Positive", "Lower power tariffs could reduce operational costs for cement companies, potentially boosting profitability", 0.4], ["Installation of two capital-intensive nuclear power plants", "Negative", "Increased focus on nuclear power may lead to higher electricity costs, affecting cement manufacturing costs", 0.3], ["Expand cooperation in the areas of trade and competitiveness", "Positive", "Increased trade and competitiveness could lead to higher demand for cement in infrastructure projects, potentially boosting cement sector growth", 0.4], ["Public Private Partnership", "Positive", "Enhanced public-private partnerships could drive infrastructure development, increasing the demand for cement", 0.5], ["Domestic Resource Mobilization", "Positive", "Improved domestic resource mobilization could lead to more funds for infrastructure projects, potentially benefiting the cement industry", 0.4], ["Short supply of vaccines", "Negative", "The scarcity of vaccines indicates potential challenges in meeting demand, which could impact various industries reliant on a fully vaccinated workforce, including the cement sector", 0.3], ["Non-production of vaccines in Pakistan", "Negative", "Lack of local vaccine production could slow down the economic recovery, potentially affecting sectors such as construction that are sensitive to economic conditions", 0.4], ["US ban on raw material exports", "Negative", "Restrictions on raw material exports could create supply chain disruptions, affecting industries reliant on steady material flow, including cement production", 0.5], ["Ramping up production in the short run is very difficult", "Negative", "Challenges in quickly increasing production could extend the duration of the pandemic, negatively impacting industries like construction that rely on stable economic conditions", 0.2], ["Pharma firms are incentivised to take up vaccine and drug research through public financial support", "Positive", "Government incentives for research can accelerate vaccine production, potentially leading to faster economic recovery and benefiting industries such as cement", 0.4], ["Tensions between Pakistan and India seem to be easing", "Positive", "Easing tensions could lead to improved cross-border trade, benefiting the cement industry by reducing tariffs and logistical challenges", 0.4], ["Announced an agreement for strict observance of all agreements, understandings and cease firing along the LoC", "Positive", "Stability along the border could reduce disruptions in supply chains, positively impacting cement production and distribution", 0.5], ["Discuss reinstatement, at the earliest, of high commissioners in both the countries", "Positive", "Restoration of diplomatic ties may enhance economic cooperation, potentially benefiting the cement sector through increased bilateral trade", 0.3], ["Precipitous withdrawal would create a vacuum for spoilers", "Negative", "Instability in Afghanistan could lead to regional instability, potentially affecting trade routes and increasing costs for cement imports and exports", 0.4], ["Instability in Afghanistan could be detrimental for the peace and stability of the entire region", "Negative", "Regional instability may result in disruptions in supply chains, impacting the cement industry by increasing transportation costs and affecting overall demand", 0.5], ["Anti-peace elements such as al-Qaeda and Daish could also take benefit of the possible chaos", "Negative", "Heightened security risks in the region could lead to increased operational costs and delays for cement companies, especially those involved in cross-border trade", 0.3], ["The Ghani-led government had refused the United States plan to bring an interim government", "Negative", "Ongoing political instability and lack of a clear transitional government in Afghanistan may affect regional economic conditions, potentially impacting demand for cement and related infrastructure projects", 0.4], ["Ring Road Project in Rawalpindi", "Negative", "Concerns or propaganda against the project could signal potential delays or disruptions in related infrastructure projects, affecting the demand for cement in construction", 0.3], ["Billion Tree Tsunami has won international acclaim", "Positive", "Environmental initiatives like the Billion Tree Tsunami project could lead to increased demand for construction materials such as cement due to related infrastructure developments", 0.3], ["Billion-Tree Tsunami project for environment protection", "Positive", "Environmental initiatives like the Billion-Tree Tsunami project may increase demand for construction materials, including cement, due to related infrastructure developments", 0.3], ["Construction of the new campus of Sir Syed University will commence with the construction of the mosque at 200 acres Land of Sir Syed University at Education City", "Positive", "The construction of a new campus and mosque may drive demand for construction materials, including cement, potentially benefiting the cement sector", 0.4], ["Ban on all kinds of indoor and outdoor gatherings", "Negative", "Restrictions on public gatherings may lead to decreased construction activities and reduced demand for cement, negatively impacting the cement sector", 0.4], ["Enforcement of expanded lockdowns with effect from March 29", "Negative", "Expanded lockdowns could disrupt construction projects and lower cement consumption, adversely affecting the cement sector", 0.4], ["Increasing taxes and hiking power and gas tariffs", "Negative", "Higher operational costs due to increased power and gas tariffs could negatively impact the cement sector's profitability", 0.4], ["Withdrawal of corporate tax exemptions to the tune of Rs140 billion", "Negative", "The removal of corporate tax exemptions could increase the tax burden on cement companies, affecting their financial stability and investment attractiveness", 0.4], ["Protecting nature increases our vulnerability to pandemics, accelerates climate change, and threatens food security", "Negative", "Accelerated climate change and environmental degradation could impact the cement industry by affecting raw material availability and increasing operational costs", 0.3], ["We need to unite and take urgent action now to set nature on the path to recovery and secure a nature-positive world", "Negative", "Efforts to address environmental issues may lead to stricter regulations on industrial activities, including the cement sector, potentially increasing operational costs", 0.3], ["The bill on SBP's autonomy aims at targeting inflation rather than economic growth", "Negative", "Focus on inflation control could lead to higher interest rates, potentially increasing borrowing costs for the cement sector", 0.3], ["The proposed changes include giving up the government’s right to borrow from the central bank", "Negative", "Restrictions on government borrowing from the SBP could lead to increased reliance on commercial banks, potentially crowding out private sector borrowing including for cement industry investments", 0.4], ["The bill includes extending the SBP governor’s tenure from three to five years to minimize political interference", "Negative", "Longer tenure for the SBP governor may result in stricter monetary policies, potentially affecting interest rates and borrowing costs for the cement industry", 0.3], ["Opponents argue that the changes could free the central bank from accountability and focus solely on price stabilization", "Negative", "The shift in focus could lead to higher interest rates and reduced attention to economic growth, impacting the cement sector's financial conditions", 0.4], ["The session would centre around giving 'absolute autonomy' to the State Bank of Pakistan (SBP)", "Negative", "Increased autonomy for the SBP could lead to tighter monetary policy, which might raise interest rates and impact borrowing costs for the cement industry", 0.3], ["The important changes include extending the tenure of the SBP governor to minimize political interference", "Negative", "Reduced political interference could lead to more stringent monetary policies, potentially affecting the cost of borrowing for the cement sector", 0.3], ["The bill aims at targeting inflation rather than economic growth", "Negative", "Focus on inflation control might result in higher interest rates, which could negatively impact the cement industry’s financial conditions", 0.3], ["PRGMEA urged the government to lower the cost of production", "Positive", "Lower production costs for the textile sector could indirectly benefit the cement industry by reducing competition for resources and stabilizing overall economic conditions", 0.2], ["PRGMEA suggested that cotton yarn and fabric should be exempted from all duties and taxes", "Negative", "Although specific to textiles, such exemptions could potentially shift government focus away from supporting other industries, including cement", 0.1], ["Proposed abolition of taxes and duties on import of polyester staple fibre", "Negative", "Similar policy changes in other sectors could lead to increased competition for government incentives and resources, potentially impacting the cement sector", 0.1], ["The amendments seek to make the SBP more autonomous", "Positive", "Enhanced autonomy of the SBP could lead to more stable economic conditions, indirectly benefiting the cement sector by reducing economic volatility", 0.3], ["Proposed amendments intend to redefine SBP's mandate with domestic price stability as the primary objective", "Positive", "A focus on price stability may lead to lower inflation, reducing the cost pressures on cement manufacturers", 0.4], ["Discontinuation of quasi-fiscal operations by SBP", "Positive", "Reducing quasi-fiscal operations may lead to more focused monetary policies, potentially benefiting the cement sector by creating a more stable economic environment", 0.3], ["The Rashakai Special Economic Zone (REZ) is developing rapidly", "Positive", "Development of the REZ may increase demand for construction materials, including cement, due to infrastructure projects", 0.6], ["Chinese state-owned Century Steel Company has started building infrastructure", "Positive", "Increased industrial activity at Rashakai can boost demand for cement", 0.5], ["Rashakai SEZ will create 200,000 employment opportunities and boost economic activities", "Positive", "Economic growth and increased construction activities in the SEZ can lead to higher cement consumption", 0.6], ["Enhancing trade between Pakistan and the US", "Positive", "Improved trade relations could lead to increased economic activity and infrastructure projects in Pakistan, potentially benefiting the cement sector", 0.4], ["Pakistan's inclination towards China and CPEC", "Positive", "Ongoing support for CPEC may lead to increased infrastructure development, boosting demand for cement", 0.5], ["Debt servicing and interest payments prevent the country from using its resources for development of infrastructure", "Negative", "Insufficient investment in infrastructure could delay or reduce construction projects, impacting cement demand and sector growth", 0.6], ["Pakistan has the lowest development ranking in the region and stands at 110 out of 137 countries in terms of infrastructure", "Negative", "Poor infrastructure development can lead to decreased construction activity, reducing demand for cement and affecting sector performance", 0.7], ["The government is forced to increase taxes on business and the general public", "Negative", "Increased taxes can reduce business profitability and spending power, potentially leading to decreased investment in construction and cement", 0.5], ["Disturbing news about ratings agencies downgrading sovereign ratings", "Negative", "A downgrade in sovereign ratings can lead to higher borrowing costs and reduced investor confidence, impacting infrastructure development and cement demand", 0.6], ["China Pakistan Economic Corridor (CPEC) is a potential positive", "Positive", "CPEC may drive infrastructure development and construction activity, increasing demand for cement", 0.7], ["USD buying rate at 155.00", "Negative", "A higher buying rate for USD could increase costs for cement manufacturers importing raw materials or machinery, affecting their profitability", 0.5], ["Euro buying rate at 182.90", "Negative", "A higher buying rate for EUR could lead to increased costs for imports from Europe, impacting the cost structure for cement producers", 0.5], ["AED buying rate at 42.21", "Negative", "An increase in the buying rate for AED may raise costs for transactions in markets where the AED is used, potentially affecting cement industry expenses", 0.4], ["SAR buying rate at 41.37", "Negative", "A higher buying rate for SAR could increase costs for transactions in Saudi Arabia, impacting cement sector costs", 0.4], ["USD 6M bill buying rate at 154.44", "Negative", "A higher rate for USD bills could indicate increased borrowing costs, which may affect the financing conditions for cement companies", 0.4], ["EUR 6M bill buying rate at 182.63", "Negative", "A higher rate for EUR bills may impact the cost of imports from Europe, potentially increasing production costs for cement manufacturers", 0.4], ["USD selling rate at 157.06 and buying rate at 153.45", "Negative", "A stronger US dollar could increase the cost of imports, including raw materials for cement production, which may negatively impact the cement sector", 0.3], ["EUR selling rate at 185.01 and buying rate at 181.06", "Negative", "A stronger Euro could affect the cost of imported materials from Europe, potentially raising production costs for cement companies", 0.3],

["Bank Holiday 1st Ramazan: Wednesday 14th April 2021 (Subject to sight of moon)", "Neutral", "The holiday may cause temporary delays in market operations but is not directly related to the cement industry or its stock index", 0.1], ["Due to Bank Holiday on April 14, 2021, settlement shall be merged on April 15th 2021 for trades of 12th and 13th April 2021", "Neutral", "The merging of settlement dates due to the holiday is an operational change and does not directly impact the cement sector", 0.1], ["Pak rupee appreciated its worth in the process of trading against the greenback in the local currency market", "Positive", "An appreciation of the local currency (PKR) can reduce the cost of importing raw materials for cement production, potentially benefiting the cement sector", 0.5], ["PKR gained 50 paisas for both buying and selling over yesterday’s rates", "Positive", "An appreciation of PKR against USD can decrease the cost of imported inputs for the cement industry, positively impacting the sector", 0.5], ["PKR gained 5 paisas for buying and 10 paisas for selling against UAE Dirham", "Positive", "A stronger PKR against other currencies can lower costs for cement manufacturers relying on imports, benefiting the sector", 0.4], ["The value of the U.S. dollar in terms of the SDR is the reciprocal of the sum of the dollar values, based on market exchange rates", "Negative", "A strengthening SDR value could indicate a stronger local currency, potentially raising the cost of imports for the cement industry", 0.3], ["The value in terms of the SDR of each of the other currencies shown above is derived from that currency's representative exchange rate against the U.S. dollar", "Negative", "Changes in the SDR value against the U.S. dollar could influence the cost of foreign raw materials, potentially impacting the cement sector", 0.3], ["The dollar was traded at around Rs154.50 during the day but closed at Rs154.59 compared to Rs155.01 a day before", "Positive", "A weaker US dollar can reduce the cost of importing raw materials, potentially benefiting the cement industry", 0.3], ["The greenback has depreciated 7.5 per cent against the rupee since August 2020", "Positive", "A significant depreciation of the dollar can lower the cost of foreign imports, including materials for cement production", 0.4], ["The demand for dollars from importers had slowed down", "Positive", "Reduced demand for dollars indicates lower import costs, which can be favorable for cement manufacturers who rely on imported materials", 0.3], ["Exporters have been selling their dollar holdings in large numbers reducing prices", "Positive", "Increased availability of rupees and lower dollar prices can benefit industries by reducing the cost of imports", 0.3], ["During Ramazan and Eid season overseas Pakistanis used to send more money to their families", "Positive", "Higher inflows of remittances can strengthen the local currency, indirectly supporting sectors reliant on imported materials, such as cement", 0.2], ["Spot Rate decreased by Rs 300 per maund to Rs 11,900 per maund", "Negative", "A decrease in cotton spot rates could indicate broader market trends and impact industries reliant on raw materials, such as cement, by affecting overall market confidence", 0.2], ["Depreciating dollar slashed profit margins for exporters", "Positive", "A weaker dollar could lead to cheaper imports, potentially lowering costs for industries including cement if they rely on imported raw materials", 0.3], ["Higher availability of raw material at cheaper rates in domestic market", "Positive", "Increased availability of cheaper raw materials could reduce production costs for industries including cement, leading to potentially positive impacts on the sector", 0.3], ["Imported cotton would be cheaper with falling greenback rates", "Positive", "Lower costs for imported raw materials could benefit the cement sector if similar trends apply to their inputs, improving cost-efficiency and potentially boosting margins", 0.3], ["Local individuals' net buying of Rs 519,863,303", "Positive", "Increased net buying by local individuals could signal greater confidence in the market, potentially benefiting sectors including cement, as it might indicate higher investment or spending", 0.3], ["Foreign investors' net selling of Rs -304,974,918", "Negative", "Significant net selling by foreign investors could reflect reduced confidence or capital flight, which might negatively impact stock market indices, including those related to the cement sector", 0.3], ["Local corporates' net selling of Rs -214,888,385", "Negative", "Net selling by local corporates could indicate lower corporate confidence or liquidity concerns, which may have a ripple effect on the cement sector and other related industries", 0.3], ["Removal of additional customs duty and regulatory duty on synthetic yarns", "Negative", "While this specifically concerns the textile industry, the broader principle of reducing import duties on raw materials could potentially influence similar policies for the cement sector, indirectly impacting production costs and competitiveness", 0.2], ["Rise in prices of raw materials and shortage affecting textile production costs", "Negative", "Increased raw material prices and shortages could indicate broader inflationary trends, which might also impact the cost structure for cement production", 0.2], ["Weekly inflation for the combined group in the period ended on March 25 slightly increased by 0.61 per cent", "Negative", "Rising inflation can lead to increased costs for raw materials and operational expenses for cement manufacturers, potentially reducing profitability", 0.3], ["Increase in prices of essential food items including tomatoes 20.04pc, chicken 8.81pc, bread 1.64pc", "Negative", "Although not directly related, rising prices of essential items could indicate broader inflationary pressures that might also affect costs for cement production", 0.2], ["Electricity was most expensive in Pakistan compared to the other countries in the world", "Negative", "High electricity costs can increase operational expenses for cement manufacturers, potentially reducing profitability", 0.3], ["The price of electricity would be increased up to Rs35 per unit next month", "Negative", "An increase in electricity prices could further raise operational costs for cement companies, negatively impacting their financial performance", 0.4], ["Pakistan has kick-started the process of cutting port charges to bring down the cost of imports and exports", "Positive", "Reduced port charges could lower transportation costs for cement manufacturers, potentially improving their profitability and competitive edge", 0.3], ["The major port charges include port dues, pilotage in, pilotage out, pilotage charges, ship berthing and storage", "Positive", "Rationalizing port charges may decrease shipping costs, which could benefit cement producers by lowering their overall operational expenses", 0.3], ["Local traders alone pay an estimated $5-6 billion in freight charges, including port charges, for import and export in a year", "Positive", "Reducing freight and port charges could indirectly lower costs for cement companies involved in import and export activities, potentially enhancing their financial performance", 0.2], ["The cost of doing business has gone up in the country following increase in power tariff", "Negative", "Increased operational costs due to higher power tariffs may negatively impact cement manufacturers' profit margins", 0.3], ["International freight charges had gone up by around 500-700% since June 2020", "Negative", "Significant increases in international freight charges may have increased costs for cement exporters, potentially affecting their competitiveness and profitability", 0.4], ["Hike in energy tariff", "Negative", "Increased energy tariffs will raise production costs for cement manufacturers, negatively impacting profitability", 0.8], ["Discontinuation of gas supply to factories generating their own electricity", "Negative", "Discontinuation of gas supply may force factories to rely on more expensive electricity, raising production costs and affecting the cement sector", 0.8], ["Difference between captive power plants and K-Electric rates", "Negative", "Higher electricity costs from K-Electric compared to captive power plants can increase operating expenses for cement manufacturers", 0.7], ["Removal of gas connections to Karachi industries", "Negative", "Removal of gas connections may disrupt production for cement plants in Karachi, increasing operational costs", 0.7], ["Increase in electricity rates", "Negative", "Increased electricity rates could lead to higher production costs for cement manufacturers, potentially impacting profitability", 0.7], ["Increase in power prices", "Negative", "Higher power prices will increase production costs, negatively affecting the cement sector's profitability", 0.7], ["Increase in fuel prices", "Negative", "Higher fuel prices contribute to increased transportation and production costs for cement, potentially impacting overall sector performance", 0.6], ["The coronavirus pandemic had hit closer to home, the first case had just surfaced", "Negative", "The pandemic's impact on the overall economic environment could potentially decrease construction activities and affect cement demand", 0.3], ["The third wave of the virus came and they could not take chances with people’s lives", "Negative", "Continued concerns about the pandemic might lead to further restrictions or delays in construction projects, negatively impacting cement consumption", 0.3], ["The pandemic has devastated economic growth in the West and the countries that rely on it", "Negative", "Economic downturns in key markets can lead to reduced demand for cement, affecting the sector's performance", 0.4], ["It’s going to create a certain degree of poverty across the world", "Negative", "Increased poverty could lead to reduced investment in infrastructure projects, negatively impacting cement industry growth", 0.3], ["The pandemic has changed things in other ways as well", "Negative", "Uncertain changes in economic conditions due to the pandemic could affect the cement industry's market stability", 0.3], ["The 300MW coal-based power project at Gwadar was also discussed.", "Negative", "A delay or changes in the coal-based power project could impact the availability and cost of energy, indirectly affecting the cement sector's operational costs and profitability", 0.3], ["The board advised PPIB to expedite the project as it is important to develop the backward areas of Gwadar, which will be the future economic hub of the country.", "Positive", "Expedited development of the Gwadar project could stimulate economic activity in the region, potentially increasing demand for construction materials including cement", 0.4], ["The board also granted extension in Financial Closing Date of 330 MW imported coal power project of M/s Siddiqsons Energy Limited (SEL).", "Negative", "Extensions in financial closure could delay energy project completions, potentially affecting the energy supply and costs for industries including cement", 0.3], ["Development of Rashakai SEZ being carried out expeditiously", "Positive", "Expedited development of SEZ can boost industrial activity, including construction projects that may increase demand for cement", 0.3], ["Aggressive marketing of Rashakai SEZ to local and foreign investors", "Positive", "Effective marketing can attract investment in the SEZ, potentially increasing construction projects and cement demand", 0.3], ["Custom and duty exemption on capital goods for SEZs", "Positive", "Exemptions can reduce costs for industries in SEZ, including cement manufacturers, potentially benefiting the sector", 0.3], ["Elimination of 1.5% turnover tax for SEZs", "Positive", "Tax elimination can reduce operational costs for industries in SEZ, including cement manufacturers, potentially boosting sector growth", 0.3], ["Domestic companies showing keen interest in setting up enterprises in Rashakai SEZ", "Positive", "Increased interest from domestic companies may lead to more construction projects, increasing demand for cement", 0.3], ["Investment of $79 million to produce 500,000 tons of Rebar/Construction steel in Rashakai SEZ", "Positive", "Increased production of construction steel in the SEZ could boost demand for cement, benefiting the cement sector", 0.4], ["Investment of $77 million for production of Industrial Steel in Rashakai SEZ", "Positive", "Increased industrial steel production could lead to higher demand for cement in construction projects, benefiting the cement sector", 0.4], ["10MW electricity available for Rashakai SEZ, with additional 160MW under construction", "Positive", "Improved electricity supply to SEZ can support industrial operations, including cement production, potentially benefiting the sector", 0.3], ["Gas supply of 30MMcfd to Rashakai SEZ underway", "Positive", "Reliable gas supply can support industrial processes in SEZ, including cement production, potentially benefiting the sector", 0.3], ["Development work of Rashakai SEZ being carried out at a fast pace", "Neutral", "Not directly related to the cement sector", 0.0], ["Aggressive marketing of Rashakai SEZ to local and foreign investors", "Neutral", "Not directly related to the cement sector", 0.0], ["Rashakai SEZ has potential to become a hub of economic activity", "Neutral", "Not directly related to the cement sector", 0.0], ["Approval of custom and duty exemption on capital goods for SEZs", "Positive", "Custom and duty exemptions could reduce costs for cement manufacturing equipment in SEZs, potentially benefiting the cement sector", 0.3], ["Elimination of 1.5 percent turnover tax for SEZs", "Positive", "Removal of turnover tax could lower operational costs for businesses in SEZs, including cement companies", 0.3], ["Domestic companies showing keen interest in setting up enterprises in Rashakai SEZ", "Neutral", "Not directly related to the cement sector", 0.0], ["Development of Karachi Coastal Comprehensive Development Zone", "Neutral", "Not directly related to the cement sector", 0.0], ["Government approved Rs1 billion in advertisement budget to highlight its economic performance", "Neutral", "Not directly related to the cement sector", 0.0], ["IMF revived Pakistan’s bailout package", "Neutral", "Not directly related to the cement sector", 0.0], ["Pakistan agreed to impose over Rs700 billion in new taxes from July", "Negative", "New taxes could increase costs for cement companies and potentially affect their profitability", 0.3], ["36% increase in power tariffs", "Negative", "Higher electricity costs could increase operational expenses for cement producers", 0.4], ["Inflation expected to remain between 7.9% and 9.5% next month", "Negative", "High inflation may lead to increased costs for raw materials and operational expenses in the cement sector", 0.3], ["Government’s policy measures causing inflationary pressure", "Negative", "Inflationary pressure could negatively impact the cement sector by increasing costs and reducing demand", 0.3], ["International commodity prices rising, especially oil and food prices", "Neutral", "Not directly related to the cement sector", 0.0], ["Increase in electricity prices to Rs5.65 per unit in six months", "Negative", "Higher electricity prices will likely increase production costs for cement manufacturers", 0.4], ["Downside risk to cotton production impacting agriculture sector", "Neutral", "Not directly related to the cement sector", 0.0], ["The Pakistani currency has maintained uptrend against the US dollar", "Positive", "A stronger rupee may reduce the cost of imported raw materials for cement production, potentially benefiting the sector", 0.4], ["The World Bank approved $1.3 billion for Pakistan, including $600 million for Ehsaas programme", "Neutral", "Not directly related to the cement sector", 0.0], ["Pakistan geared up to raise up to $1 billion from international market through launch of the Eurobond", "Neutral", "Not directly related to the cement sector", 0.0], ["IMF loan programme worth $6 billion", "Neutral", "Not directly related to the cement sector", 0.0], ["Revival of IMF programme has revived the confidence level of local and global business community in the domestic economy", "Positive", "Improved economic confidence can boost investments and demand in the cement sector", 0.3], ["World Bank and Asian Development Bank (ADB) both have announced to give $10-12 billion each over the next five years", "Neutral", "Not directly related to the cement sector", 0.0], ["Pakistan’s foreign currency reserves further increased by $275 million", "Neutral", "Not directly related to the cement sector", 0.0], ["Reserves may continue to grow with increasing inflows under the State Bank of Pakistan’s (SBP) initiative Roshan Digital Account (RDA) and strong remittances sent home by overseas Pakistanis", "Neutral", "Not directly related to the cement sector", 0.0], ["SBP’s sole mandate would be “price control and to independently set monetary policy and exchange rate policy”", "Negative", "Higher interest rates due to tighter monetary policy could impact borrowing costs for cement manufacturers, potentially affecting their expansion plans and profitability", 0.2], ["The government shall not borrow or seek guarantees from the SBP for its operations", "Positive", "Reduced government borrowing could alleviate pressure on interest rates, potentially benefiting the cement sector", 0.2], ["The proposed amendments seek protection to actions taken in good faith by providing indemnity", "Neutral", "Not directly related to the cement sector", 0.0], ["The National Accountability Bureau (NAB) and the Federal Investigation Agency (FIA) cannot investigate the SBP governor, deputy governors, its executives and board and committee members without permission of its board of directors", "Neutral", "Not directly related to the cement sector", 0.0], ["The definition of “monetary liabilities” has been introduced", "Neutral", "Not directly related to the cement sector", 0.0], ["SBP’s board of directors to conduct its business free from political and bureaucratic interventions", "Neutral", "Not directly related to the cement sector", 0.0]]



# Create a DataFrame
df = pd.DataFrame(data, columns=["Article", "Sentiment", "Relevance", "Score"])

# Save the DataFrame to a CSV file
csv_filename = "/content/news_articles.csv"
df.to_csv(csv_filename, index=False)

# Download the CSV file
files.download(csv_filename)
